<a href="https://colab.research.google.com/github/wTatiya/scrapping_hrms/blob/main/scrapping_hrms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# dependencies
!apt-get update > /dev/null
!apt install chromium-chromedriver > /dev/null
!pip install selenium schedule openpyxl > /dev/null

import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from tqdm import tqdm

# Config browser
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

#  login
driver.get("https://hrms129.thai-nrls.org/hrms13754/Account/Login")

#  credent
username = "hrmsqsh"
password = "qsh13754"

driver.find_element(By.ID, "txtUserName").send_keys(username)
driver.find_element(By.ID, "txtPass").send_keys(password)
driver.find_element(By.ID, "btnLogon").click()

# data page
time.sleep(2)
driver.get("https://hrms129.thai-nrls.org/hrms13754/Database/RiskBookingAllList")
time.sleep(2)

# 🧹 Scraper + Save Function
import pandas as pd
import re
import smtplib
from email.message import EmailMessage
import schedule
import time

def run_scraper_and_save():
    print("🔍 Starting scrape...")
    data = []
    max_pages = 250  # safeguard

    for _ in range(max_pages):
        time.sleep(1.5)
        rows = driver.find_elements(By.CSS_SELECTOR, "#tbDataList tbody tr")

        if not rows:
            cards = driver.find_elements(By.CSS_SELECTOR, ".ibox-content h3")
            if not cards:
                print("⚠️ No data found on this page.")
                break
            page_text = driver.find_element(By.CSS_SELECTOR, ".ibox-content").text
            data.append([page_text])
            break

        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            if cols:
                data.append([col.text.strip() for col in cols])

        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next"]')
            if "disabled" in next_btn.get_attribute("class"):
                print("✅ Reached last page.")
                break
            next_btn.click()
        except Exception:
            print("✅ No more pages or next button missing.")
            break

    # --- Convert to clean DataFrame ---
    df_raw = pd.DataFrame(data)
    df_raw.columns = ["Incident_ID", "Incident_Type", "Location", "Unused1", "Severity_Code", "Status_Info"]

    # Extract fields
    df_raw["Incident_Type_Code"] = df_raw["Incident_Type"].str.extract(r"^([A-Z]+\d+):")
    df_raw["Incident_Type_Details"] = df_raw["Incident_Type"].str.extract(r"^[A-Z]+\d+:(.*)")

    def extract_date(label, text):
        match = re.search(rf"{label}\s*:\s*(\d{{2}}/\d{{2}}/\d{{4}})", text)
        return pd.to_datetime(match.group(1), format="%d/%m/%Y", errors="coerce") if match else pd.NaT

    for col, label in [
        ("Incident_Date", "วันที่เกิดเหตุ"),
        ("Discovery_Date", "วันที่ค้นพบ"),
        ("Report_Date", "วันที่บันทึกรายงาน"),
        ("Confirmation_Date", "วันที่ยืนยัน"),
        ("Notification_Date", "วันที่แจ้งเหตุ"),
        ("Status_Date", "วันที่ของสถานะ"),
        ("Resolution_Date", "วันที่กลุ่ม/หน่วยงานหลักแก้ไขเสร็จ"),
    ]:
        df_raw[col] = df_raw["Status_Info"].apply(lambda x: extract_date(label, x))

    # Harm level logic
    def classify_harm(row):
        code = row["Severity_Code"].strip()
        clinical = general = ""
        if code in ["A", "B"]:
            clinical = "ไม่เกิดความรุนแรง (No Harm)"
        elif code in ["C", "D"]:
            clinical = "เกิดความรุนแรงน้อย (Low Harm)"
        elif code in ["E", "F"]:
            clinical = "เกิดความรุนแรงปานกลาง (Moderate Harm)"
        elif code in ["G", "H"]:
            clinical = "เกิดความรุนแรงมาก (Severe Harm)"
        elif code == "I":
            clinical = "เสียชีวิต (Death)"
        elif code == "1":
            general = "น้อยมาก"
        elif code == "2":
            general = "น้อย"
        elif code == "3":
            general = "ปานกลาง"
        elif code == "4":
            general = "สูง"
        elif code == "5":
            general = "สูงมาก"
        return pd.Series([clinical, general])

    df_raw[["Harm_Level_Clinical", "Harm_Level_General"]] = df_raw.apply(classify_harm, axis=1)

    # Final clean table
    df_final = df_raw[
        [
            "Incident_ID", "Incident_Type_Code", "Incident_Type_Details", "Location", "Severity_Code",
            "Harm_Level_Clinical", "Harm_Level_General", "Incident_Date", "Discovery_Date", "Report_Date",
            "Confirmation_Date", "Notification_Date", "Status_Date", "Resolution_Date"
        ]
    ]

    # Save to Excel
    excel_path = "/mnt/data/risk_data_cleaned.xlsx"
    df_final.to_excel(excel_path, index=False)
    print(f"💾 Saved cleaned Excel to {excel_path}")
    return excel_path


# ✉️ Email Sending Function
def send_excel_via_email(sender_email, app_password, recipient_email, file_path):
    msg = EmailMessage()
    msg["Subject"] = "📄 Daily Risk Report Export"
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg.set_content("Attached is the daily risk report exported automatically.")

    with open(file_path, "rb") as f:
        file_data = f.read()
        file_name = f.name.split("/")[-1]
        msg.add_attachment(
            file_data,
            maintype="application",
            subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            filename=file_name,
        )

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(sender_email, app_password)
        smtp.send_message(msg)

    print(f"📧 Email sent to {recipient_email}")


# 🕒 Schedule Job
def job():
    print("🚀 Running scheduled job...")
    path = run_scraper_and_save()
    send_excel_via_email(
        sender_email="tatiya1641@gmail.com",
        app_password="ospc ewzy ccfr titk",  # Gmail app password
        recipient_email="tatiya1641@gmail.com",
        file_path=path,
    )

# Schedule for 08:00 daily
schedule.every().day.at("08:00").do(job)

# 👇 Run once now (for testing)
job()

# Keep alive (for continuous daily mode)
while True:
    schedule.run_pending()
    time.sleep(60)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


🚀 Running scheduled job...
🔍 Starting scrape...
✅ No more pages or next button missing.


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [ ]:
from google.colab import files
files.download("risk_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>